# Finding overlapping compounds

This notebook aims to identify overlapping compounds present in both the `cell_injury` and `JUMP` datasets. These overlapping compounds will be used for subsetting the `JUMP` dataset, which we'll consider as the ground truth for subsequent analyses.

## Approach
1. **Identifying Overlapping Compounds**: We compare the compounds present in both datasets to identify the overlapping ones.
2. **Subsetting the JUMP Dataset**: Once the overlapping compounds are identified, we subset the `JUMP` dataset to include only those compounds, forming our ground truth dataset.
3. **Save dataset**: The dataset will be saved in the `./results/3.jump_analysis`


In [1]:
import pathlib
import pandas as pd

In [4]:
jump_data_path = pathlib.Path(
    "../../data/JUMP_data/JUMP_all_plates_normalized_negcon.csv.gz"
)
df = pd.read_csv(jump_data_path)

/tmp/ipykernel_342403/3358820507.py:2: DtypeWarning: Columns (6,7,5800,5801,5802,5804) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(jump_data_path)


In [6]:
[name for name in df.columns.tolist() if name.startswith("Metadata_")]

['Metadata_broad_sample',
 'Metadata_Plate',
 'Metadata_Well',
 'Metadata_gene',
 'Metadata_pert_type',
 'Metadata_control_type',
 'Metadata_target_sequence',
 'Metadata_negcon_control_type',
 'Metadata_solvent',
 'Metadata_InChIKey',
 'Metadata_pert_iname',
 'Metadata_pubchem_cid',
 'Metadata_smiles']

In [15]:
df.Metadata_pubchem_cid.unique()

array([       nan, 9.8839e+06, 6.7900e+02, 2.1620e+03, 3.6060e+03,
       9.4175e+04, 4.6189e+07, 6.8844e+04, 2.4826e+07, 5.9397e+07,
       1.1315e+07, 3.2950e+03, 9.8882e+06, 4.8175e+05, 1.0297e+07,
       9.9573e+06, 1.1844e+07, 5.3276e+04, 5.1228e+05, 5.7430e+03,
       1.4000e+03, 7.2837e+07, 5.9174e+07, 1.3657e+06, 1.6154e+07,
       5.3277e+06, 2.5110e+07, 6.5724e+05, 8.3163e+05, 4.0370e+03,
       4.4130e+07, 1.1637e+07, 1.3280e+06, 4.4483e+07, 1.0963e+06,
       2.4905e+07, 2.4800e+07, 5.2824e+06, 2.3662e+07, 4.7879e+06,
       3.4060e+03, 1.0811e+05, 6.5866e+04, 4.3076e+06, 4.6742e+07,
       4.6886e+07, 1.2618e+06, 4.4250e+07, 1.1180e+07, 1.1351e+07,
       1.0027e+07, 4.4621e+07, 5.3328e+07, 9.9385e+06, 1.3252e+05,
       1.0809e+05, 5.7522e+07, 2.4827e+07, 5.7022e+06, 1.4749e+06,
       1.2035e+04, 3.0041e+06, 1.7270e+03, 2.6209e+05, 2.1200e+03,
       2.5227e+07, 5.3112e+06, 1.7754e+07, 6.9147e+06, 2.1610e+03,
       9.8010e+06, 1.6655e+07, 1.6231e+04, 5.7343e+07, 1.5117e